# ViT(Vision Transformer)를 이용한 이미지 분류
## Introduction
CIFAR100 데이터셋에 대해서 이미지 분류 작업을 ViT(Vision Transformer)를 이용하여 적용해 본다. ViT 모델은 convolution layer를 사용하지 않고 이미지 패치의 시퀀스를 self-attention하는 Transformer 구조를 적용한다.

In [1]:
# Tensorflow는 2.4 이상버전으로,
# 거기에 조금 특별한 패키지인 Tensorflow Addons를 필요로 한다.
# !pip install -U tensorflow-addons

In [2]:
# Setup : Import packages
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [4]:
# 데이터 준비
num_classes = 100
input_shape = (32, 32, 3)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

169017344/169001437 [==============================] - 419s 2us/step
x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [13]:
# Hyperparameter setting
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
image_size = 72 # 입력 이미지를 이 크기로 크기를 재조정한다.
patch_size = 6 # 입력 이미지에서 추출한 패치의 크기
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
] # transformer 층들의 크기
transformer_layers = 8
mlp_head_units = [2048, 1024] # 마지막 classifier의 dense layer의 크기

In [14]:
# Apply data augmentation
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# 학습 데이터의 평균과 분산을 normalizaiton(정규화)를 위해 계산한다.
data_augmentation.layers[0].adapt(x_train)

In [15]:
# MLP(multilayer Perceptron) 구현한다.
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [16]:
# Patch를 만든는것은 layer로 구현한다.
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
    
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = "VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [17]:
# 예시 이미지의 패치들을 확인해 본다.
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i+1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

Image size: 72 X 72
Patch size: 6 X 6
Patches per image: 144
Elements per patch: 108


In [18]:
# patch를 인코딩하는 layer들을 구현한다.
# PatchEncoder layer는 projection_dim의 크기의 벡터로 패치를 투영함으로써 선형적으로 변형시킬 것이다.
# 추가적으로, 투영된 벡터에 학습가능한 position embedding을 더한다.
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim = num_patches, output_dim=projection_dim
        )
        
    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

## ViT 모델을 만들어본다.
ViT 모델은 여러개의 Transformer 블록들로 구성되며, 각각 `layers.MultiHeadAttention layer`를 사용하며, 패치들의 시퀀스에 self-attention 매커니즘을 적용한다. Transformer 블록들은 `[batch_size, num_patches, projection_dim]` 텐서를 만들며, 이것은 softmax연산을 활용한 classifier head를 통해서 마지막 클래스 분류 출력을 만들어낸다. <br>
<br>
ViT 논문에서 말하는 기술에서는 학습가능한 embedding을 이미지 표현에 대해 제공되는 인코딩된 패치들의 시퀀스에 제공한다. 그와 다르게, 마지막 Transformer 블록의 모든 출력은 `layers.Flatten()`을 이용하여 reshape된다. 그리고 이것은 분류기의 입력에 쓰이는 이미지 표현으로 사용된다.
<br>
`layers.GlobalAveragePooling1D` layer는 Transformer 블록의 출력을 모으는것 대신 사용될 수 있다. 특히, 패치의 수나 사영되는 차원의 수가 클때 사용된다.

In [19]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
# 컴파일, 학습, 모드의 평가
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

100번의 epoch이 지난 후, ViT 모델은 test 데이터에 대해서 55, 82%의 정확도를 기록했다. 이것은 ResNet50V2가 같은 CIFAR100 데이터에서 67%의 정확도를 기록한 것에 비교해서는 좋은 결과가 아니다. <br>
<br>
논문에서 기록했다는 좋은 성과는 ViT 모델을 JFT-300M 데이터셋에 대해 사전학습을 하고, 타겟 데이터셋에 대해 fine-tuning함으로써 기록할 수 있다. 사전 학습 없이 모델 성능을 향상시키기 위해서, 모델을 더 많은 epoch으로 학습시키거나 더 많은Transformer layer들을 이용하거나 입력 이미지의 크기를 재조정하거나, 패치 크기를 바꾸거나, 투영 차원의 크기를 증가시킨다. 또한, 논문에 언급되었듯이 모델의 성능은 모델 구조의 선택 뿐만 아니라, 학습률 계획, optimizer, weight decay 등 파라미터 설정에 대해서도 영향을 받는다. 실제로는, 더 크고, 더 높은 해상도의 데이터셋을 이용하여 사전학습한 ViT 모델을 fine-tune 하기를 추천한다.